# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 9:40AM,241633,10,0085857112,ISDIN Corporation,Released
1,Jul 12 2022 9:40AM,241633,10,0085857049,ISDIN Corporation,Released
2,Jul 12 2022 9:40AM,241633,10,0085857048,ISDIN Corporation,Released
3,Jul 12 2022 9:40AM,241633,10,0085857078,ISDIN Corporation,Released
4,Jul 12 2022 9:40AM,241633,10,0085857361,ISDIN Corporation,Released
5,Jul 12 2022 9:40AM,241633,10,0085857362,ISDIN Corporation,Released
6,Jul 12 2022 9:40AM,241633,10,0085857390,ISDIN Corporation,Released
7,Jul 12 2022 9:40AM,241633,10,0085857389,ISDIN Corporation,Released
8,Jul 12 2022 9:40AM,241633,10,0085857426,ISDIN Corporation,Released
9,Jul 12 2022 9:40AM,241633,10,0085857486,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,241629,Released,3
44,241630,Released,11
45,241632,Released,41
46,241633,Released,15
47,241634,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241629,NaN,NaN,3.0
241630,NaN,NaN,11.0
241632,NaN,NaN,41.0
241633,NaN,NaN,15.0
241634,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241507,0.0,10.0,29.0
241508,0.0,0.0,57.0
241510,0.0,3.0,48.0
241511,0.0,15.0,1.0
241517,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241507,0,10,29
241508,0,0,57
241510,0,3,48
241511,0,15,1
241517,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241507,0,10,29
1,241508,0,0,57
2,241510,0,3,48
3,241511,0,15,1
4,241517,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241507,,10,29
1,241508,,,57
2,241510,,3,48
3,241511,,15,1
4,241517,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation
15,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation
16,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation
57,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation
104,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc."
115,Jul 12 2022 9:17AM,241629,12,Hush Hush
118,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc."
119,Jul 12 2022 9:17AM,241627,12,Hush Hush
129,Jul 12 2022 9:14AM,241625,20,Alumier Labs Inc.
148,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,,,15
1,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,,,1
2,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,,,41
3,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,,,47
4,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",,,11
5,Jul 12 2022 9:17AM,241629,12,Hush Hush,,,3
6,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc.",,,1
7,Jul 12 2022 9:17AM,241627,12,Hush Hush,,,10
8,Jul 12 2022 9:14AM,241625,20,Alumier Labs Inc.,,,19
9,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15,,
1,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1,,
2,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,41,,
3,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,47,,
4,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",11,,
5,Jul 12 2022 9:17AM,241629,12,Hush Hush,3,,
6,Jul 12 2022 9:17AM,241628,19,"GCH Granules USA, Inc.",1,,
7,Jul 12 2022 9:17AM,241627,12,Hush Hush,10,,
8,Jul 12 2022 9:14AM,241625,20,Alumier Labs Inc.,19,,
9,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15,,
1,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1,,
2,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,41,,
3,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,47,,
4,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15.0,NaN,NaN
1,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1.0,NaN,NaN
2,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,41.0,NaN,NaN
3,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,47.0,NaN,NaN
4,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:40AM,241633,10,ISDIN Corporation,15.0,0.0,0.0
1,Jul 12 2022 9:40AM,241634,10,ISDIN Corporation,1.0,0.0,0.0
2,Jul 12 2022 9:29AM,241632,10,ISDIN Corporation,41.0,0.0,0.0
3,Jul 12 2022 9:24AM,241622,10,ISDIN Corporation,47.0,0.0,0.0
4,Jul 12 2022 9:18AM,241630,20,"Exact-Rx, Inc.",11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5314693,658.0,78.0,0.0
12,483256,13.0,0.0,0.0
16,483118,2.0,0.0,0.0
19,1449562,4.0,12.0,17.0
20,966377,59.0,13.0,0.0
21,483225,2.0,0.0,0.0
22,241623,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5314693,658.0,78.0,0.0
1,12,483256,13.0,0.0,0.0
2,16,483118,2.0,0.0,0.0
3,19,1449562,4.0,12.0,17.0
4,20,966377,59.0,13.0,0.0
5,21,483225,2.0,0.0,0.0
6,22,241623,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,658.0,78.0,0.0
1,12,13.0,0.0,0.0
2,16,2.0,0.0,0.0
3,19,4.0,12.0,17.0
4,20,59.0,13.0,0.0
5,21,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,658.0
1,12,Released,13.0
2,16,Released,2.0
3,19,Released,4.0
4,20,Released,59.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,17.0,0.0,0.0,0.0
Executing,78.0,0.0,0.0,12.0,13.0,0.0,0.0
Released,658.0,13.0,2.0,4.0,59.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,78.0,0.0,0.0,12.0,13.0,0.0,0.0
2,Released,658.0,13.0,2.0,4.0,59.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,78.0,0.0,0.0,12.0,13.0,0.0,0.0
2,Released,658.0,13.0,2.0,4.0,59.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()